In [ ]:
import pandas as pd
import numpy as np
import os
import os.path as osp

import torchio as tio
from pathlib import Path
from scipy.interpolate import PchipInterpolator, interp1d
from matplotlib import pyplot as plt
from scipy.stats import norm

import statsmodels.stats.inter_rater as irr
from statsmodels.stats.inter_rater import aggregate_raters

In [ ]:
internal_data_pths = [
                      './datasets/LN_classify/Fudan_HN_LN_22-23_all/Fudan_HN_LN_20231204_patches',
                      ]

external_data_pths = ['./datasets/LN_classify/Fudan_HN_LN_22-23_all/CGMH/CGMH_2024_patches',
                      './datasets/LN_classify/Fudan_HN_LN_22-23_all/TCGA/TCGA-HNSC_selected_patches',
                      './datasets/LN_classify/Fudan_HN_LN_22-23_all/CGMH_Oral/CGMH_Oral_patches'
                      ]

name_to_sad = {}
for data_idx, crop_pth in enumerate(internal_data_pths + external_data_pths):
    cropfile = osp.join(crop_pth, "cropping_list.csv")
    df = pd.read_csv(cropfile)
    for idx, row in df.iterrows():
        name_to_sad[row['basename'].lower()] = float(row['recist'])

In [ ]:
_MIN_SAD = 5.0
# _MAX_SAD = {'EENT': 21.8850, 'CGMH': 11.25, 'TCGA': 9999, 'CGMH-Oral': 9999, 'Dev': 9999} # q50 limit
_MAX_SAD = {'EENT': 9999, 'CGMH': 9999, 'TCGA': 9999, 'CGMH-Oral': 9999, 'Dev': 9999} # no limit
renji_ln_count = {}
renji_ln_rm = {}
cgmh_ln_count = {}
cgmh_ln_rm = {}
tcga_ln_count = {}
tcga_ln_rm = {}

# renji_pth = "./datasets/LN_classify/Fudan_HN_LN_22-23_all/Fudan_HN_LN_20231204/2023-08_LN_data_reorganize_external"
renji_pth = "./datasets/LN_classify/Fudan_HN_LN_22-23_all/Fudan_HN_LN_20231204_patches/Ext"
# cgmg_pth = "./datasets/LN_classify/Fudan_HN_LN_22-23_all/CGMH/CGMH_2024_reoriented"
cgmh_pth = "./datasets/LN_classify/Fudan_HN_LN_22-23_all/CGMH/CGMH_2024_patches"
# tcga_pth = "./datasets/LN_classify/Fudan_HN_LN_22-23_all/TCGA/TCGA-HNSC_selected_reori_labeled"
tcga_pth = "./datasets/LN_classify/Fudan_HN_LN_22-23_all/TCGA/TCGA-HNSC_selected_patches"

for p in Path(renji_pth).rglob("*_mask.nii.gz"):
    basename = osp.basename(p).replace("_mask.nii.gz", "")
    patient = basename.split('_ins')[0]
    ln_id = int(basename.split('_ins')[-1].split('_')[0])
    if '_pos' in basename:
        sad = name_to_sad[basename.lower().split('_pos')[0]]
    else:
        sad = name_to_sad[basename.lower().split('_neg')[0]]
    
    if patient not in renji_ln_count:
        renji_ln_count[patient] = []
        renji_ln_rm[patient] = []
    
    if sad >= 5 or '_pos' in basename:
        renji_ln_count[patient].append(ln_id)
    else:
        renji_ln_rm[patient].append(ln_id)

for p in Path(cgmh_pth).rglob("*_mask.nii.gz"):
    basename = osp.basename(p).replace("_mask.nii.gz", "")
    patient = basename.split('_ins')[0]
    ln_id = int(basename.split('_ins')[-1].split('_')[0])
    if '_pos' in basename:
        sad = name_to_sad[basename.lower().split('_pos')[0]]
    else:
        sad = name_to_sad[basename.lower().split('_neg')[0]]
    
    if patient not in cgmh_ln_count:
        cgmh_ln_count[patient] = []
        cgmh_ln_rm[patient] = []
    
    if sad >= 5 or '_pos' in basename:
        cgmh_ln_count[patient].append(ln_id)
    else:
        cgmh_ln_rm[patient].append(ln_id)
    
for p in Path(tcga_pth).rglob("*_mask.nii.gz"):
    basename = osp.basename(p).replace("_mask.nii.gz", "")
    patient = basename.split('_ins')[0]
    ln_id = int(basename.split('_ins')[-1].split('_')[0])
    if '_pos' in basename:
        sad = name_to_sad[basename.lower().split('_pos')[0]]
    else:
        sad = name_to_sad[basename.lower().split('_neg')[0]]
    
    if patient not in tcga_ln_count:
        tcga_ln_count[patient] = []
        tcga_ln_rm[patient] = []
    
    if sad >= 5 or '_pos' in basename:
        tcga_ln_count[patient].append(ln_id)
    else:
        tcga_ln_rm[patient].append(ln_id)

#### Load annotations

In [ ]:
renji_anno_pth = "./datasets/LN_classify/Fudan_HN_LN_22-23_all/Fudan_HN_LN_20231204/headNeck_LN_label_infor(7).csv"
cgmh_anno_pth = "./datasets/LN_classify/Fudan_HN_LN_22-23_all/CGMH/headNeck_LN_label_infor.csv"
tcga_anno_pth = "./datasets/LN_classify/Fudan_HN_LN_22-23_all/TCGA/headNeck_LN_label_infor.csv"
df_renji = pd.read_csv(renji_anno_pth)
df_cgmh = pd.read_csv(cgmh_anno_pth)
df_tcga = pd.read_csv(tcga_anno_pth)

In [ ]:
# Load reader 1 and reader 2's annotation
rd1_renji_pth = "./datasets/LN_classify/Fudan_HN_LN_22-23_all/ReaderStudy/reader1_renji_remapped.csv"
rd1_cgmh_pth = "./datasets/LN_classify/Fudan_HN_LN_22-23_all/ReaderStudy/reader1_cgmh.csv"
rd1_tcga_pth = "./datasets/LN_classify/Fudan_HN_LN_22-23_all/ReaderStudy/reader1_tcga_remapped.csv"
df_rd1_renji = pd.read_csv(rd1_renji_pth)
df_rd1_cgmh = pd.read_csv(rd1_cgmh_pth)
df_rd1_tcga = pd.read_csv(rd1_tcga_pth)

rd2_renji_pth = "./datasets/LN_classify/Fudan_HN_LN_22-23_all/ReaderStudy/reader2_renji_remapped.csv"
rd2_cgmh_pth = "./datasets/LN_classify/Fudan_HN_LN_22-23_all/ReaderStudy/reader2_cgmh.csv"
rd2_tcga_pth = "./datasets/LN_classify/Fudan_HN_LN_22-23_all/ReaderStudy/reader2_tcga_remapped.csv"
df_rd2_renji = pd.read_csv(rd2_renji_pth)
df_rd2_cgmh = pd.read_csv(rd2_cgmh_pth)
df_rd2_tcga = pd.read_csv(rd2_tcga_pth)

rd3_renji_pth = "./datasets/LN_classify/Fudan_HN_LN_22-23_all/ReaderStudy/reader3_sn_renji_remapped.csv"
rd3_cgmh_pth = "./datasets/LN_classify/Fudan_HN_LN_22-23_all/ReaderStudy/reader3_sn_cgmh.csv"
rd3_tcga_pth = "./datasets/LN_classify/Fudan_HN_LN_22-23_all/ReaderStudy/reader3_sn_tcga_remapped.csv"
df_rd3_renji = pd.read_csv(rd3_renji_pth)
df_rd3_cgmh = pd.read_csv(rd3_cgmh_pth)
df_rd3_tcga = pd.read_csv(rd3_tcga_pth)

rd4_renji_pth = "./datasets/LN_classify/Fudan_HN_LN_22-23_all/ReaderStudy/reader4_dd_renji_remapped.csv"
rd4_cgmh_pth = "./datasets/LN_classify/Fudan_HN_LN_22-23_all/ReaderStudy/reader4_dd_cgmh.csv"
rd4_tcga_pth = "./datasets/LN_classify/Fudan_HN_LN_22-23_all/ReaderStudy/reader4_dd_tcga_remapped.csv"
df_rd4_renji = pd.read_csv(rd4_renji_pth)
df_rd4_cgmh = pd.read_csv(rd4_cgmh_pth)
df_rd4_tcga = pd.read_csv(rd4_tcga_pth)

rd5_renji_pth = "./datasets/LN_classify/Fudan_HN_LN_22-23_all/ReaderStudy/reader5_xbb_renji_remapped.csv"
rd5_cgmh_pth = "./datasets/LN_classify/Fudan_HN_LN_22-23_all/ReaderStudy/reader5_xbb_cgmh.csv"
rd5_tcga_pth = "./datasets/LN_classify/Fudan_HN_LN_22-23_all/ReaderStudy/reader5_xbb_tcga_remapped.csv"
df_rd5_renji = pd.read_csv(rd5_renji_pth)
df_rd5_cgmh = pd.read_csv(rd5_cgmh_pth)
df_rd5_tcga = pd.read_csv(rd5_tcga_pth)

rd6_renji_pth = "./datasets/LN_classify/Fudan_HN_LN_22-23_all/ReaderStudy/reader6_albert_renji_remapped.csv"
rd6_cgmh_pth = "./datasets/LN_classify/Fudan_HN_LN_22-23_all/ReaderStudy/reader6_albert_cgmh.csv"
rd6_tcga_pth = "./datasets/LN_classify/Fudan_HN_LN_22-23_all/ReaderStudy/reader6_albert_tcga_remapped.csv"
df_rd6_renji = pd.read_csv(rd6_renji_pth)
df_rd6_cgmh = pd.read_csv(rd6_cgmh_pth)
df_rd6_tcga = pd.read_csv(rd6_tcga_pth)

In [13]:
def generate_rater_results(df_reader, df_anno, dataset):
    rater_met_res = []
    rater_ene_res = []
    read_order = []

    for idx, reader_row in df_reader.iterrows():
        patient = reader_row["file_name"].replace(".nii.gz", "")
        ln_id2idx = {}
        if dataset == 'renji':
            patient_annotations = len(renji_ln_count[patient])
            all_lns = set(renji_ln_count[patient])
            sorted_ids = sorted(renji_ln_count[patient])
            for idx, ln_id in enumerate(sorted_ids):
                ln_id2idx[ln_id] = idx
        elif dataset == 'cgmh':
            if 'CGMH_Larynx20240117_anon_025' in patient:
                continue
            patient_annotations = len(cgmh_ln_count[patient])
            all_lns = set(cgmh_ln_count[patient])
            sorted_ids = sorted(cgmh_ln_count[patient])
            for idx, ln_id in enumerate(sorted_ids):
                ln_id2idx[ln_id] = idx
        elif dataset == 'tcga':
            patient_annotations = len(tcga_ln_count[patient])
            all_lns = set(tcga_ln_count[patient])
            sorted_ids = sorted(tcga_ln_count[patient])
            for idx, ln_id in enumerate(sorted_ids):
                ln_id2idx[ln_id] = idx
        else:
            raise ValueError("Invalid type")
            
        row_meta_preds = reader_row["positive_LN_id"]
        if isinstance(row_meta_preds, pd.Series):
            row_meta_preds = row_meta_preds.values[0]
        
        if isinstance(row_meta_preds, float) and np.isnan(row_meta_preds):
            meta_preds = []
        elif isinstance(row_meta_preds, (int, float)):
            meta_preds = [int(row_meta_preds)]
        elif isinstance(row_meta_preds, str) and row_meta_preds != "无转移" and row_meta_preds != "":
            meta_preds = [int(p) for p in row_meta_preds.rstrip(',').split(",")]
        elif row_meta_preds == "无转移" or row_meta_preds == "":
            meta_preds = []
        else:
            raise ValueError("No predicted found")
        
        # remove not selected LNs (e.g., <5mm negative and >LIMIT ENEs)
        if dataset == 'renji':
            meta_preds = [p for p in meta_preds if p not in renji_ln_rm[patient]]
        elif dataset == 'cgmh':
            meta_preds = [p for p in meta_preds if p not in cgmh_ln_rm[patient]]
        elif dataset == 'tcga':
            meta_preds = [p for p in meta_preds if p not in tcga_ln_rm[patient]]
        
        row_ene_preds = reader_row["ENE_LN_id"]
        if isinstance(row_ene_preds, pd.Series):
            row_ene_preds = row_ene_preds.values[0]
          
        if isinstance(row_ene_preds, float) and np.isnan(row_ene_preds):
            ene_preds = []
        elif isinstance(row_ene_preds, (int, float)):
            ene_preds = [int(row_ene_preds)]
        elif isinstance(row_ene_preds, str):
            ene_preds = [int(p) for p in row_ene_preds.rstrip(',').split(",")]
        else:
            ene_preds = []
        
        # remove not selected LNs (e.g., <5mm negative and >LIMIT ENEs)
        if dataset == 'renji':
            ene_preds = [p for p in ene_preds if p not in renji_ln_rm[patient]]
        elif dataset == 'cgmh':
            ene_preds = [p for p in ene_preds if p not in cgmh_ln_rm[patient]]
        elif dataset == 'tcga':
            ene_preds = [p for p in ene_preds if p not in tcga_ln_rm[patient]]
        
        anno_row = df_anno[df_anno["file_name"] == patient]
        if anno_row.empty:
            print("{} is not matched any annotation".format(patient))
            continue
        
        row_anno_meta = anno_row["positive_LN_id"]
        if isinstance(row_anno_meta, pd.Series):
            row_anno_meta = row_anno_meta.values[0]
        
        if isinstance(row_anno_meta, float) and np.isnan(row_anno_meta):
            anno_meta = []
        elif isinstance(row_anno_meta, (int, float)):
            anno_meta = [int(row_anno_meta)]
        elif isinstance(row_anno_meta, str) and row_anno_meta != "无转移":
            anno_meta = [int(p) for p in row_anno_meta.rstrip(',').split(",")]
        elif row_anno_meta == "无转移":
            anno_meta = []
        else:
            raise ValueError("No annotation found")
        
        # remove not selected LNs (e.g., <5mm negative and >LIMIT ENEs)
        if dataset == 'renji':
            anno_meta = [p for p in anno_meta if p not in renji_ln_rm[patient]]
        elif dataset == 'cgmh':
            anno_meta = [p for p in anno_meta if p not in cgmh_ln_rm[patient]]
        elif dataset == 'tcga':
            anno_meta = [p for p in anno_meta if p not in tcga_ln_rm[patient]]
        
        row_anno_ene = anno_row["ENE_LN_id"]
        if isinstance(row_anno_ene, pd.Series):
            row_anno_ene = row_anno_ene.values[0]
          
        if isinstance(row_anno_ene, float) and np.isnan(row_anno_ene):
            anno_ene = []
        elif isinstance(row_anno_ene, (int, float)):
            anno_ene = [int(row_anno_ene)]
        elif isinstance(row_anno_ene, str):
            anno_ene = [int(p) for p in row_anno_ene.rstrip(',').split(",")]
        else:
            anno_ene = []
        
        # remove not selected LNs (e.g., <5mm negative and >LIMIT ENEs)
        if dataset == 'renji':
            anno_ene = [p for p in anno_ene if p not in renji_ln_rm[patient]]
        elif dataset == 'cgmh':
            anno_ene = [p for p in anno_ene if p not in cgmh_ln_rm[patient]]
        elif dataset == 'tcga':
            anno_ene = [p for p in anno_ene if p not in tcga_ln_rm[patient]]

            
        meta_pred_idx = set([ln_id2idx[p] for p in meta_preds if p in all_lns])
        ene_preds_idx = set([ln_id2idx[p] for p in ene_preds if p in all_lns])

        # if len(all_lns) < max(all_lns):
        #     a = 0
        # anno_meta = set(anno_meta)
        # anno_ene = set(anno_ene)
        
        # prepare for inter-rater reliability
        read_order.append(patient)
        meta_read_res = np.zeros(patient_annotations).astype(int)
        ene_read_res = np.zeros(patient_annotations).astype(int)
        meta_read_res[list(meta_pred_idx)] = 1
        ene_read_res[list(ene_preds_idx)] = 1

        rater_met_res.append(meta_read_res)
        rater_ene_res.append(ene_read_res)
    
    ordered_rater_met = [rater_met_res[i] for i in np.argsort(read_order)]
    ordered_rater_ene = [rater_ene_res[i] for i in np.argsort(read_order)]
    rater_met_res = np.concatenate(ordered_rater_met, axis=0)
    rater_ene_res = np.concatenate(ordered_rater_ene, axis=0)

    return rater_met_res, rater_ene_res


In [15]:
# EENT
renji_r1_met_rate, renji_r1_ene_rate = generate_rater_results(df_rd1_renji, df_renji, dataset='renji')
renji_r2_met_rate, renji_r2_ene_rate = generate_rater_results(df_rd2_renji, df_renji, dataset='renji')
renji_r4_met_rate, renji_r4_ene_rate = generate_rater_results(df_rd4_renji, df_renji, dataset='renji')
renji_r5_met_rate, renji_r5_ene_rate = generate_rater_results(df_rd5_renji, df_renji, dataset='renji')
renji_r6_met_rate, renji_r6_ene_rate = generate_rater_results(df_rd6_renji, df_renji, dataset='renji')

renji_met_rater_all = np.stack([renji_r1_met_rate, renji_r2_met_rate, renji_r4_met_rate, renji_r5_met_rate, renji_r6_met_rate], axis=1)
print(renji_met_rater_all.shape)
intermediate = aggregate_raters(renji_met_rater_all)
renji_met_fleiss_kappa = irr.fleiss_kappa(intermediate[0], "fleiss")
print("Fudan EENT Meta Fleiss Kappa: ", renji_met_fleiss_kappa)

renji_ene_rater_all = np.stack([renji_r1_ene_rate, renji_r2_ene_rate, renji_r4_ene_rate, renji_r5_ene_rate, renji_r6_ene_rate], axis=1)
intermediate = aggregate_raters(renji_ene_rater_all)
renji_ene_fleiss_kappa = irr.fleiss_kappa(intermediate[0], "fleiss")
print("Fudan EENT ENE Fleiss Kappa: ", renji_ene_fleiss_kappa)


(395, 5)
Fudan EENT Meta Fleiss Kappa:  0.43365783895228965
Fudan EENT ENE Fleiss Kappa:  0.500185585099204


In [16]:
# CGMH
cgmh_r1_met_rate, cgmh_r1_ene_rate = generate_rater_results(df_rd1_cgmh, df_cgmh, dataset='cgmh')
cgmh_r2_met_rate, cgmh_r2_ene_rate = generate_rater_results(df_rd2_cgmh, df_cgmh, dataset='cgmh')
cgmh_r4_met_rate, cgmh_r4_ene_rate = generate_rater_results(df_rd4_cgmh, df_cgmh, dataset='cgmh')
cgmh_r5_met_rate, cgmh_r5_ene_rate = generate_rater_results(df_rd5_cgmh, df_cgmh, dataset='cgmh')
cgmh_r6_met_rate, cgmh_r6_ene_rate = generate_rater_results(df_rd6_cgmh, df_cgmh, dataset='cgmh')


cgmh_met_rater_all = np.stack([cgmh_r1_met_rate, cgmh_r2_met_rate, cgmh_r4_met_rate, cgmh_r5_met_rate, cgmh_r6_met_rate], axis=1)
print(cgmh_met_rater_all.shape)
intermediate = aggregate_raters(cgmh_met_rater_all)
cgmh_met_fleiss_kappa = irr.fleiss_kappa(intermediate[0], "fleiss")
print("CGMH Meta Fleiss Kappa: ", cgmh_met_fleiss_kappa)

cgmh_ene_rater_all = np.stack([cgmh_r1_ene_rate, cgmh_r2_ene_rate, cgmh_r4_ene_rate, cgmh_r5_ene_rate, cgmh_r6_ene_rate], axis=1)
intermediate = aggregate_raters(cgmh_ene_rater_all)
cgmh_ene_fleiss_kappa = irr.fleiss_kappa(intermediate[0], "fleiss")
print("CGMH ENE Fleiss Kappa: ", cgmh_ene_fleiss_kappa)


(194, 5)
CGMH Meta Fleiss Kappa:  0.36882068758894443
CGMH ENE Fleiss Kappa:  0.332389878742885


In [17]:
# TCGA
tcga_r1_met_rate, tcga_r1_ene_rate = generate_rater_results(df_rd1_tcga, df_tcga, dataset='tcga')
tcga_r2_met_rate, tcga_r2_ene_rate = generate_rater_results(df_rd2_tcga, df_tcga, dataset='tcga')
tcga_r4_met_rate, tcga_r4_ene_rate = generate_rater_results(df_rd4_tcga, df_tcga, dataset='tcga')
tcga_r5_met_rate, tcga_r5_ene_rate = generate_rater_results(df_rd5_tcga, df_tcga, dataset='tcga')
tcga_r6_met_rate, tcga_r6_ene_rate = generate_rater_results(df_rd6_tcga, df_tcga, dataset='tcga')

tcga_met_rater_all = np.stack([tcga_r1_met_rate, tcga_r2_met_rate, tcga_r4_met_rate, tcga_r5_met_rate, tcga_r6_met_rate], axis=1)
print(tcga_met_rater_all.shape)
intermediate = aggregate_raters(tcga_met_rater_all)
tcga_met_fleiss_kappa = irr.fleiss_kappa(intermediate[0], "fleiss")
print("TCGA-TCIA Meta Fleiss Kappa: ", tcga_met_fleiss_kappa)

# tcga_ene_rater_all = np.stack([tcga_r1_ene_rate, tcga_r2_ene_rate, tcga_r4_ene_rate, tcga_r5_ene_rate, tcga_r6_ene_rate], axis=1)
# intermediate = aggregate_raters(tcga_ene_rater_all)
# tcga_ene_fleiss_kappa = irr.fleiss_kappa(intermediate[0], "fleiss")
# print("TCGA-TCIA ENE Fleiss Kappa: ", tcga_ene_fleiss_kappa)


(210, 5)
TCGA-TCIA Meta Fleiss Kappa:  0.3691068018289795


#### Cohen's Kappa

In [ ]:
from sklearn.metrics import cohen_kappa_score

In [ ]:
renji_annotations = renji_met_rater_all.transpose()
num_annotators = renji_annotations.shape[0]

# List to store pairwise Cohen's Kappa scores
kappa_scores = []

# Calculate Cohen's Kappa for each pair of annotators
for i in range(num_annotators):
    for j in range(i + 1, num_annotators):
        kappa = cohen_kappa_score(renji_annotations[i], renji_annotations[j])
        kappa_scores.append(kappa)

# Calculate the average Cohen's Kappa
average_kappa = np.mean(kappa_scores)

print(f'EENT pairwise Cohen\'s Kappa scores: {kappa_scores}')
print(f'EENT Average Cohen\'s Kappa: {average_kappa}')
